<a href="https://colab.research.google.com/github/suyogpradhan0/Spatial-Data-Management-with-Google-Earth-Engine/blob/main/The%20DEM%20of%20Nepal%20and%20cloud-free%20of%20Sentinel-2%20imagery%20of%20the%20year%202021%20for%20Nepal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-414/blob/master/book/labs/lab_03.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).


## Question 1

Find a DEM dataset in the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/), clip the DEM to your home state, and display the DEM with a proper color palette on the map. For example, the sample map below shows the DEM of the state of Colorado.

In [1]:
# %pip install pygis

In [2]:
import ee
import geemap

In [3]:
geemap.ee_initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=8u5FgnjWGloxIn57qyJOoP-enWYnqGUkP_mUMLYBv2w&tc=JAeUqCvCZHNRmMn225kbpNlakjT_LPzZnsP6tpVFAuM&cc=AaA4FrvelpW-E-nxMdJ7-DNHFL2cl5Q3ZNAklEIuujw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AeaYSHBx0LofuGWYeEdubQvakvMi3dLsl07MNifrlr1Zv3oziIbW7J5WPV0

Successfully saved authorization token.


In [4]:
# Create a Map object
Map = geemap.Map()

# Define the region of interest (ROI) for Nepal
nepal = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', 'Nepal'))

# Load a Digital Elevation Model (DEM) dataset
dem = ee.Image('USGS/SRTMGL1_003')

# Clip the DEM to Nepal
dem_nepal = dem.clip(nepal)

# Add the clipped DEM to the map with a proper color palette
Map.addLayer(dem_nepal, {
    'min': 0,
    'max': 8000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}, 'DEM')

# Center the map around Nepal
Map.centerObject(nepal, 7)

#Add text to the map
text = "Created by Suyog Pradhan"
Map.add_text(text, fontsize=20, position='bottomright')

# Display the map
Map

Map(center=[28.26765258698865, 83.9761416224727], controls=(WidgetControl(options=['position', 'transparent_bg…

![](https://i.imgur.com/OLeSt7n.png)

## Question 2

Use [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/sentinel-2) or [Landsat-9 data](https://developers.google.com/earth-engine/datasets/catalog/landsat-9), create a cloud-free imagery of the year 2021 for your home state. Display the imagery on the map with a proper band combination. For example, the sample map below shows a cloud-free false-color composite of Sentinel-2 imagery of the year 2021 for the state of Colorado.

In [6]:
# Create a Map object
Map = geemap.Map()

# Define the region of interest (ROI) for Nepal
nepal = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', 'Nepal'))

# Function to mask clouds using the Sentinel-2 QA band
def mask_clouds(image):
    # Select the QA60 band from Sentinel-2
    QA60 = image.select(['QA60'])
    # Create a mask by combining cloud and cirrus flags
    cloud_mask = QA60.bitwiseAnd(1 << 10).eq(0).And(QA60.bitwiseAnd(1 << 11).eq(0))
    # Update the image mask with the cloud mask
    return image.updateMask(cloud_mask)

# Define the time range for the imagery (e.g., the year 2021)
start_date = '2021-01-01'
end_date = '2021-12-31'

# Load Sentinel-2 imagery for the specified time range and region
sentinel_collection = (ee.ImageCollection('COPERNICUS/S2')
                       .filterBounds(nepal)
                       .filterDate(ee.Date(start_date), ee.Date(end_date))
                       .map(mask_clouds))

# Select bands for visualization (e.g., RGB)
visualization_bands = ['B4', 'B3', 'B2']

# Create a mosaic of the imagery
mosaic = sentinel_collection.mosaic()

# Clip the mosaic to the Nepal boundary
mosaic_nepal = mosaic.clip(nepal)

# Add the cloud-free imagery to the map with the specified band combination
Map.addLayer(mosaic_nepal, {'bands': visualization_bands, 'min': 0, 'max': 8000}, 'Sentinel-2 2021')

# Center the map around the specified ROI
Map.centerObject(nepal, 7)

#Add text to the map
text = "Created by Suyog Pradhan"
Map.add_text(text, fontsize=20, position='bottomright')

# Display the map
Map

Map(center=[28.26765258698865, 83.9761416224727], controls=(WidgetControl(options=['position', 'transparent_bg…

![](https://i.imgur.com/wD0VamV.png)